In [5]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [4]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

In [3]:
result_list = convert_to_df('../persistent_data/lstm.json')
result_list

,true_positives,false_positives,true_negatives,false_negatives,alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,precision_with_cfa,precision_with_syscalls,scenario,ngram,batch_size,embedding_size,return_value,thread_change_flag,time_delta,detection_time
0,172,0,1227918,194417,172,174,0.988506,0,0,0.988506,1.000000,1.000000,CVE-2017-7529/,4,1024,6,True,True,False,15.711918
1,274,29,2670371,2095219,51,98,0.520408,26,3,0.520408,0.637500,0.637500,Bruteforce_CWE-307/,4,1024,6,True,True,False,50.891472
2,0,1,3396126,1474597,0,155,0.000000,1,0,0.000000,0.000000,0.000000,CVE-2012-2122/,4,1024,6,True,True,False,47.202630
3,172,0,1226104,194417,172,174,0.988506,0,0,0.988506,1.000000,1.000000,CVE-2017-7529/,6,1024,6,True,True,False,16.018071
4,0,1,2833009,285051,0,100,0.000000,1,0,0.000000,0.000000,0.000000,CVE-2014-0160/,6,1024,6,True,True,False,34.263841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,564,14,15571956,1815734,101,103,0.980583,13,1,0.980583,0.878261,0.878261,PHP_CWE-434/,6,1024,8,True,True,False,176.142924
187,166,19,12744090,1104285,98,98,1.000000,19,0,1.000000,0.837607,0.837607,CVE-2019-5418/,8,1024,8,True,True,False,127.150144
188,1801,1,13496612,1755783,137,137,1.000000,1,0,1.000000,0.992754,0.992754,CVE-2018-3760/,8,1024,8,True,True,False,169.928648
189,1018,7,15570239,1815280,103,103,1.000000,7,0,1.000000,0.936364,0.936364,PHP_CWE-434/,8,1024,8,True,True,False,162.617787


In [6]:
conn = sql.connect('eval.db')
result_list.to_sql('eval', conn)

In [120]:
conn = sql.connect('eval.db')
result = pd.read_sql('SELECT \
                             ngram, \
                             embedding_size as embed, \
                             return_value as rv, \
                             thread_change_flag as tcf, \
                             time_delta as td, \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             ROUND(AVG(detection_rate),2) as DR, \
                             COUNT(*) \
                     FROM eval \
                     GROUP BY ngram, \
                              embedding_size, \
                              rv, \
                              tcf, \
                              td \
                     HAVING DR > 0.5 \
                     ORDER BY  \
                              DR desc, \
                              fp_sum desc', conn)

In [121]:
result

,ngram,embed,rv,tcf,td,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,4,8,1,1,0,20.69,1.13,349,0.85,16
1,8,8,1,1,0,14.56,0.69,244,0.73,16
2,6,6,1,0,0,8.87,0.60,142,0.68,15
3,8,6,1,1,0,23.25,0.88,386,0.67,16
4,6,8,1,1,0,8.38,0.13,136,0.63,16
5,4,6,1,1,0,12.68,1.00,260,0.61,19
6,4,6,1,0,0,16.86,0.57,244,0.54,14
7,4,8,1,0,0,12.13,0.47,189,0.54,15


In [141]:
conn = sql.connect('eval.db')
result = pd.read_sql('SELECT scenario, time_delta as td, AVG(detection_rate) \
                      FROM eval \
                      WHERE td = 1 \
                      GROUP BY scenario', conn)

In [142]:
result

,scenario,td,AVG(detection_rate)


In [122]:
error_list = convert_to_df('../persistent_data/error_log.json')
error_list

,true_positives,false_positives,true_negatives,false_negatives,alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,precision_with_cfa,precision_with_syscalls,scenario,ngram,batch_size,embedding_size,return_value,thread_change_flag,time_delta
0,0,0,0,0,0,0,0,0,0,0,0,0,EPS_CWE-434/,4,1024,6,True,False,False
1,0,0,0,0,0,0,0,0,0,0,0,0,EPS_CWE-434/,6,1024,6,True,False,False
2,0,0,0,0,0,0,0,0,0,0,0,0,EPS_CWE-434/,8,1024,6,True,False,False
3,0,0,0,0,0,0,0,0,0,0,0,0,ZipSlip/,4,1024,6,True,False,False
4,0,0,0,0,0,0,0,0,0,0,0,0,ZipSlip/,6,1024,6,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,0,0,0,0,0,0,0,0,0,0,0,0,SQL_Injection_CWE-89/,8,1024,8,True,True,True
275,0,0,0,0,0,0,0,0,0,0,0,0,ZipSlip/,8,1024,8,True,True,True
276,0,0,0,0,0,0,0,0,0,0,0,0,EPS_CWE-434/,8,1024,8,True,True,False
277,0,0,0,0,0,0,0,0,0,0,0,0,ZipSlip/,6,1024,8,True,True,False


In [124]:
conn = sql.connect('error.db')
error_list.to_sql('error_log', conn)

In [135]:
conn = sql.connect('error.db')
result = pd.read_sql('SELECT scenario, \
                             ngram, \
                             embedding_size as embed, \
                             return_value as rv, \
                             thread_change_flag as tcf, \
                             time_delta as td \
                     FROM error_log \
                     GROUP BY scenario', conn)

In [136]:
result

,scenario,ngram,embed,rv,tcf,td
0,Bruteforce_CWE-307/,4,6,1,0,1
1,CVE-2012-2122/,4,6,1,0,1
2,CVE-2014-0160/,4,6,1,0,1
3,CVE-2017-7529/,4,6,1,0,1
4,CVE-2018-3760/,4,6,1,0,1
5,CVE-2019-5418/,4,6,1,0,1
6,EPS_CWE-434/,4,6,1,0,0
7,PHP_CWE-434/,4,6,1,0,1
8,SQL_Injection_CWE-89/,4,6,1,0,1
9,ZipSlip/,4,6,1,0,0
